In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import optim
import mne
from skorch.helper import predefined_split
import os

from braindecode import EEGClassifier
from braindecode.datasets.xy import create_from_X_y
from braindecode.models import EEGInceptionMI
from braindecode.preprocessing import Preprocessor, preprocess
from braindecode.augmentation import AugmentedDataLoader
from braindecode.augmentation.transforms import GaussianNoise, ChannelsDropout, TimeReverse, SignFlip
from braindecode.preprocessing import exponential_moving_standardize
from braindecode.util import set_random_seeds

set_random_seeds(seed=20170629, cuda=False)

In [55]:
# 1. DATA LOADING AND PREPARATION
def load_your_eeg_data(
    data_path="./data/mtcaic3",
    task="MI",
    split="train",
    trial_length=1750,
    eeg_channels=["FZ", "C3", "CZ", "C4", "PZ", "PO7", "OZ", "PO8"],
    sfreq=250,
    data_fraction=1.0,
):
    """
    Load EEG data optimized for braindecode - returns full trials

    Parameters
    ----------
    data_path : str
        Path to the main data directory
    task : str
        Task type ("MI" or "SSVEP")
    split : str
        Data split ("train", "test", or "validation")
    trial_length : int
        Number of time points per trial (1750)
    eeg_channels : list
        List of EEG channel names to extract
    sfreq : float
        Sampling frequency (250 Hz)
    data_fraction : float
        Fraction of data to load (0.0 < data_fraction <= 1.0)

    Returns
    -------
    X : numpy array
        EEG data of shape (n_trials, n_channels, n_timepoints)
    y : numpy array
        Labels of shape (n_trials,)
    sfreq : float
        Sampling frequency
    """

    assert 0.0 < data_fraction <= 1.0, "data_fraction must be between 0.0 and 1.0"

    # Label mapping for motor imagery
    LABEL_TO_IDX = {"Left": 0, "Right": 1}

    # Load labels if available
    labels_file = os.path.join(data_path, f"{split}.csv")
    if os.path.exists(labels_file):
        labels_df = pd.read_csv(labels_file, usecols=["subject_id", "trial_session", "trial", "task", "label"])
        task_df = labels_df.query(f"task=='{task}'")
        has_labels = True
    else:
        # For test set without labels, create dummy entries
        task_df = []
        task_dir = os.path.join(data_path, task, split)
        for subj in os.listdir(task_dir):
            subj_dir = os.path.join(task_dir, subj)
            if not os.path.isdir(subj_dir):
                continue
            for sess in os.listdir(subj_dir):
                sess_dir = os.path.join(subj_dir, sess)
                eeg_fp = os.path.join(sess_dir, "EEGdata.csv")
                if not os.path.exists(eeg_fp):
                    continue
                for trial in range(1, 11):  # 10 trials per session
                    task_df.append({"subject_id": subj, "trial_session": int(sess), "trial": trial, "label": "Left"})  # Dummy label
        task_df = pd.DataFrame(task_df)
        has_labels = False

    # Apply data fraction sampling
    if data_fraction < 1.0:
        n_samples = int(len(task_df) * data_fraction)
        task_df = task_df.sample(n=n_samples, random_state=42).reset_index(drop=True)
        print(f"Using {data_fraction*100:.1f}% of data: {n_samples} trials")

    # Prepare data containers
    trials_data = []
    trials_labels = []
    file_cache = {}

    # Only read EEG channels - ignore validation column completely
    usecols = eeg_channels

    print(f"Loading {len(task_df)} trials from {task} {split} set...")

    skipped_trials = 0

    for _, row in task_df.iterrows():
        subj = row["subject_id"]
        sess = str(row["trial_session"])
        trial = int(row["trial"])

        # Construct file path
        fp = os.path.join(data_path, task, split, subj, sess, "EEGdata.csv")

        if not os.path.exists(fp):
            print(f"Warning: missing {fp}")
            skipped_trials += 1
            continue

        # Cache CSV file to avoid repeated reading
        if fp not in file_cache:
            try:
                file_cache[fp] = pd.read_csv(fp, usecols=usecols).values
            except Exception as e:
                print(f"Error reading {fp}: {e}")
                skipped_trials += 1
                continue

        arr = file_cache[fp]

        # Extract trial data (trials are sequential in the CSV)
        start_idx = (trial - 1) * trial_length
        end_idx = trial * trial_length

        if end_idx > len(arr):
            print(f"Warning: trial {trial} extends beyond file length in {fp}")
            skipped_trials += 1
            continue

        trial_data = arr[start_idx:end_idx]

        # No validation filtering - take all data
        # Transpose to (channels, time)
        trial_eeg = trial_data.T  # Shape: (n_channels, n_timepoints)

        trials_data.append(trial_eeg)

        # Add label
        if has_labels and row["label"] in LABEL_TO_IDX:
            trials_labels.append(LABEL_TO_IDX[row["label"]])
        else:
            trials_labels.append(0)  # Dummy label for test set

    print(f"Successfully loaded {len(trials_data)} trials, skipped {skipped_trials}")

    if len(trials_data) == 0:
        raise ValueError("No valid trials found!")

    # Convert to numpy arrays - all trials should have same length now (1750)
    X = np.stack(trials_data)  # Shape: (n_trials, n_channels, n_timepoints)
    y = np.array(trials_labels, dtype=np.int64)

    print(f"Final data shape: X={X.shape}, y={y.shape}")
    print(f"Sampling frequency: {sfreq} Hz")

    return X, y, sfreq


# Load your data
X, y, sfreq = load_your_eeg_data(data_fraction=0.1)

Using 10.0% of data: 240 trials
Loading 240 trials from MI train set...
Successfully loaded 240 trials, skipped 0
Final data shape: X=(240, 8, 1750), y=(240,)
Sampling frequency: 250 Hz


In [61]:
# 2. CREATE BRAINDECODE DATASET
ch_names = ["FZ", "C3", "CZ", "C4", "PZ", "PO7", "OZ", "PO8"]  
dataset = create_from_X_y(
    X=X,
    y=y,
    drop_last_window=False,
    sfreq=sfreq,
    window_size_samples=1125,  # 4 seconds at 250Hz
    window_stride_samples=1125,  # Non-overlapping windows
    ch_names = ch_names
)


Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs
Ready.
Creating RawArray with float64 data, n_channels=8, n_times=1750
    Range : 0 ... 1749 =      0.000 ...     6.996 secs

In [62]:
# 3. PREPROCESSING PIPELINE
preprocessors = [
    # Resample if needed (skip if already at desired frequency)
    # Preprocessor('resample', sfreq=250),
    Preprocessor("set_channel_types", mapping={ch: "eeg" for ch in ["FZ", "C3", "CZ", "C4", "PZ", "PO7", "OZ", "PO8"]}),
    Preprocessor("pick_types", eeg=True),
    # Bandpass filter for motor imagery
    Preprocessor("filter", l_freq=8.0, h_freq=30.0, fir_design="firwin"),
    # Common average reference
    Preprocessor("set_eeg_reference", ref_channels="average"),
    # Exponential moving standardization (better than simple z-score)
    Preprocessor(exponential_moving_standardize),
]

preprocess(dataset, preprocessors, n_jobs=1)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal

/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finishe

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal

/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finishe

Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters


...

/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finishe

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass 

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs)
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/preprocessing/preprocess.py:85: RuntimeWarning: The unit for channel(s) C3, C4, CZ, FZ, OZ, PO7, PO8, PZ has changed from NA to V.
  getattr(raw_or_epochs, self.fn)(**self.kwargs

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


In [63]:
# 4. TRAIN/VALIDATION SPLIT
n_train = int(0.8 * len(dataset))
train_set = torch.utils.data.Subset(dataset, range(n_train))
valid_set = torch.utils.data.Subset(dataset, range(n_train, len(dataset)))
train_valid_split = predefined_split(valid_set)

In [71]:
dataset[0][0].shape[1]

1125

In [77]:
# 5. AUGMENTATION SETUP
transforms = [
    GaussianNoise(probability=0.5, std=0.1),
    ChannelsDropout(probability=0.3, p_drop=0.1),
    TimeReverse(probability=0.3),
    SignFlip(probability=0.3),
]

# 6. MODEL SETUP
n_classes = len(np.unique(y))
n_chans = dataset[0][0].shape[0] # type: ignore
input_window_samples = dataset[0][0].shape[1] # type: ignore

model = EEGInceptionMI(  
    n_chans=n_chans,  
    n_outputs=n_classes,  
    n_times=input_window_samples,  
    sfreq=sfreq,  
)

# 7. TRAINING SETUP
clf = EEGClassifier(
    model,
    cropped=False,  # Trialwise decoding
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    train_split=train_valid_split,
    optimizer__lr=0.001,
    optimizer__weight_decay=0.01,
    batch_size=32,
    max_epochs=50,
    # Use augmented data loader
    iterator_train=AugmentedDataLoader,
    iterator_train__transforms=transforms,
    # Callbacks for monitoring
    callbacks=["accuracy"],
    device="cpu",  # Since you don't have GPU
    classes=list(range(n_classes)),
    verbose=1,
)

# 8. TRAINING
print("Starting training...")
clf.fit(train_set, y=None)

/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


Starting training...


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  ================================================================================================================================================================
  EEGInceptionMI (EEGInceptionMI)                              [1, 8, 1125]              [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 8, 1125]              [1, 8, 1125, 1]           --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 8, 1125, 1]           [1, 8, 1, 1125]           --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 8, 1, 1125]           [1, 288, 1, 1125]         --                        --
  │    └─Conv2d (conv): 2-1                                    [1, 8, 1, 1125]           [1, 288, 1, 1125]         2,592                     [1, 1]
  │    └─BatchNorm2d (bn): 2-2                                 [1, 288, 1, 1125]         [1, 288, 1, 1125]         576                       --
  ├─_ResidualModuleMI (residual_block_2): 1-17                 --                        --                        (recursive)               --
  │    └─ReLU (activation): 2-3                                [1, 288, 1, 1125]         [1, 288, 1, 1125]         --                        --
  ├─_InceptionModuleMI (initial_inception_module): 1-5         [1, 8, 1, 1125]           [1, 288, 1, 1125]         --                        --
  │    └─Conv2d (bottleneck): 2-4                              [1, 8, 1, 1125]           [1, 48, 1, 1125]          432                       [1, 1]
  │    └─ModuleList (conv_list): 2-5                           --                        --                        --                        --
  │    │    └─Conv2d (0): 3-1                                  [1, 48, 1, 1125]          [1, 48, 1, 1125]          57,648                    [1, 25]
  │    │    └─Conv2d (1): 3-2                                  [1, 48, 1, 1125]          [1, 48, 1, 1125]          172,848                   [1, 75]
  │    │    └─Conv2d (2): 3-3                                  [1, 48, 1, 1125]          [1, 48, 1, 1125]          288,048                   [1, 125]
  │    │    └─Conv2d (3): 3-4                                  [1, 48, 1, 1125]          [1, 48, 1, 1125]          403,248                   [1, 175]
  │    │    └─Conv2d (4): 3-5                                  [1, 48, 1, 1125]          [1, 48, 1, 1125]          518,448                   [1, 225]
  │    └─MaxPool2d (pooling): 2-6                              [1, 8, 1, 1125]           [1, 8, 1, 1125]           --                        [1, 25]
  │    └─Conv2d (pooling_conv): 2-7                            [1, 8, 1, 1125]           [1, 48, 1, 1125]          432                       [1, 1]
  │    └─BatchNorm2d (bn): 2-8                                 [1, 288, 1, 1125]         [1, 288, 1, 1125]         576                       --
  ├─_ResidualModuleMI (residual_block_2): 1-17                 --                        --                        (recursive)               --
  │    └─ReLU (activation): 2-9                                [1, 288, 1, 1125]         [1, 288, 1, 1125]         --                        --
  ├─ModuleList (intermediate_inception_modules_1): 1-9         --                        --                        (recursive)               --
  │    └─_InceptionModuleMI (0): 2-10                          [1, 288, 1, 1125]         [1, 288, 1, 1125]         --                        --
  │    │    └─Conv2d (bottleneck): 3-6                         [1, 288, 1, 1125]         [1, 4

In [ ]:
# 9. EVALUATION
print("Evaluating model...")
train_acc = clf.score(train_set)
valid_acc = clf.score(valid_set)

print(f"Training Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {valid_acc:.4f}")

# 10. MAKING PREDICTIONS
predictions = clf.predict(valid_set)
probabilities = clf.predict_proba(valid_set)

print("Training completed!")